# 3 - Plot

## Load all results data and compute mean & std

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

import sys
PROJ_DIR = os.path.realpath(os.path.dirname(os.path.abspath('')))
sys.path.append(os.path.join(PROJ_DIR,'src'))
import plt_configs

keys = ['correct_pairings_inv', 'correct_pairings_basX', 'tau_inv', 'tau_basX', 'spearman_inv', 'spearman_basX', 'aucs_inv', 'aucs_basX', 'spearman_exceptional_inv', 'spearman_exceptional_basX', 'tau_exceptional_inv', 'tau_exceptional_basX']

result_dict = {}

DATASET_NAME = 'avila'
MODEL_NAME = '0'
GENERATION_MODE = ''
TARGET_MEASURE = '_qmeans' #'_AttributionLocalisation' | '_TopKIntersection' | '_RelevanceRankAccuracy' | '_AUC' | 'EfficientMPRT'
SUFFIX = '' #'_EfficientMPRT'

#FIG2A, FIG3A, FIG7A & FIG8A
DATASET_NAME, MODEL_NAME, GENERATION_MODE, TARGET_MEASURE, SUFFIX = 'avila', 'mlp', '_full', '_qmeans', ''
#FIG2B, FIG3B, FIG7B
#DATASET_NAME, MODEL_NAME, GENERATION_MODE, TARGET_MEASURE, SUFFIX = 'glass', 'mlp', '_full', '_qmeans', ''
#FIG8B
#DATASET_NAME, MODEL_NAME, GENERATION_MODE, TARGET_MEASURE, SUFFIX = 'avila', 'ood-mean', '_full', '_qmeans', ''
#FIG8C
#DATASET_NAME, MODEL_NAME, GENERATION_MODE, TARGET_MEASURE, SUFFIX = 'avila', 'untrained', '_full', '_qmeans', ''

num_files = 0 # Count how many files are involved for use below

for f in os.listdir(os.path.join(PROJ_DIR, 'results')):
    if f.startswith(DATASET_NAME) and f.endswith(f'{MODEL_NAME}{GENERATION_MODE}{SUFFIX}_results{TARGET_MEASURE}.npz'):#{MODEL_NAME}{GENERATION_MODE}_localization_s_area_results{TARGET_MEASURE}.npz
        FILENAME = os.path.join(PROJ_DIR, 'results', f)
        print(FILENAME)
        num_files += 1
        with np.load(FILENAME) as data:
            for k in keys:
                d = np.expand_dims(data[k], axis=0)
                if k in result_dict:
                    result_dict[k] = np.vstack((result_dict[k], d))
                else:
                    result_dict[k] = d

for k in keys:
    result_dict[f'{k}_mean'] = np.nanmean(result_dict[k], axis=0)
    result_dict[f'{k}_std'] = np.nanstd(result_dict[k], axis=0)

# DEBUG
#print(result_dict['spearman_inv'])
#print(result_dict['spearman_inv_mean'])
#print(result_dict['spearman_basX'])
#print(result_dict['spearman_basX_mean'])

In [ ]:
boundaries = [0.5, 1, 1.5, 2, 2.5]#, 3, 3.5]
counts_by_sigma_level =  np.zeros((num_files, len(boundaries)))
filenames = []
qmean_means = []
qmean_stds = []

# Genetic datasets need the distribution parameters of the random datasets, so load all dataset counts
import json
DATA_PATH = os.path.join(PROJ_DIR,'assets','data')
with open(os.path.join(DATA_PATH, 'dataset-counts.json')) as fIn:
    datasets = json.load(fIn)

fig, ax = plt.subplots()
#plt.figure(figsize=(3,2))
current_file_num = 0
for f in os.listdir(os.path.join(PROJ_DIR, 'results')):
    if f.startswith(DATASET_NAME) and f.endswith(f'{MODEL_NAME}{GENERATION_MODE}{SUFFIX}_measures.npz'):
        FILENAME = os.path.join(PROJ_DIR, 'results', f)
        filenames.append(f)
        short_filename = f.replace("_measures.npz", "").replace(DATASET_NAME, "").replace(MODEL_NAME, "").replace("_", "").replace("-", "")
        with np.load(FILENAME) as data:
            ax.hist(data[TARGET_MEASURE[1:]],\
                    alpha = 0.5,\
                    label = f'{short_filename} qmean~N({np.mean(data[TARGET_MEASURE[1:]]):.2f},{np.std(data[TARGET_MEASURE[1:]]):.2f})',\
                    #edgecolor='black',\
                    bins = 50,\
                    range=(0,1)\
                    )
            #plt.xlim(0,1)

            if GENERATION_MODE == '_genetic':
                with np.load(FILENAME.replace('_genetic', '')) as data_ref:
                    activation_ref = data_ref['output_curves'][8][-1]
                    ax.hist(data_ref['qmeans'],\
                            alpha = 0.5,\
                            label = f'{short_filename.replace("genetic", "")} qmean~N({np.mean(data_ref["qmeans"]):.2f},{np.std(data_ref["qmeans"]):.2f}) [{activation_ref:.2f}]',\
                            bins = 50\
                            )

            exceptional_counts = []
            # Compute z-score
            if GENERATION_MODE == '_genetic':
                qmean_mean = float(datasets[DATASET_NAME][f.replace('_genetic','')]['mean'])
                qmean_std = float(datasets[DATASET_NAME][f.replace('_genetic','')]['std'])
            else:
                qmean_mean = np.mean(data[TARGET_MEASURE[1:]])
                qmean_std = np.std(data[TARGET_MEASURE[1:]])
                print(f'{qmean_mean = }')
                qmean_means.append(qmean_mean)
                qmean_stds.append(qmean_std)
            
            z_scores = ((data[TARGET_MEASURE[1:]] - qmean_mean) / qmean_std).flatten()

            # Count exceptional rankings
            for i in range(1,len(boundaries)+1):
                bottom_limit = boundaries[i-1]
                top_limit = float('inf')
                if i < len(boundaries):
                    top_limit = boundaries[i]
                fraction_count = np.sum(np.logical_and(bottom_limit<=z_scores, z_scores<top_limit).astype(float) / z_scores.shape[0])
                exceptional_counts.append(fraction_count)
                counts_by_sigma_level[current_file_num, i - 1] = fraction_count
            print(f'{short_filename} {"  |  ".join((map(lambda x: f"{x:.4f}", exceptional_counts)))}')
        current_file_num += 1
#plt.legend()
ax.set_ylim(0,3.7e6)
ax.set_title(f'{"trained" if MODEL_NAME=="0" else MODEL_NAME}')
ax.set_xlabel('q')
if MODEL_NAME == "0":
    ax.set_ylabel('# elems')
else:
    ax.set_yticklabels([])
    
plt.savefig(os.path.join(PROJ_DIR, 'results', 'plots', f'{DATASET_NAME}_{MODEL_NAME}{GENERATION_MODE}_qmean_dists.png'), dpi=300, bbox_inches='tight')
plt.show()

print(counts_by_sigma_level)
print('AVG STD:', np.mean(qmean_stds))

In [ ]:
dataset_counts = {} if DATASET_NAME not in datasets else datasets[DATASET_NAME]
for i, f in enumerate(filenames):
    dataset_counts[f] = {'counts':list(counts_by_sigma_level[i]), 'mean': str(qmean_means[i]), 'std': str(qmean_stds[i])}

datasets[DATASET_NAME] = dataset_counts

with open(os.path.join(DATA_PATH, 'dataset-counts.json'), 'w') as fOut:
    json.dump(datasets, fOut)

## Plot correct pairings

In [ ]:
#plt.figure(figsize=(3,2))
plt.fill_between(range(1,11), result_dict['correct_pairings_basX_mean']+result_dict['correct_pairings_basX_std'], result_dict['correct_pairings_basX_mean']-result_dict['correct_pairings_basX_std'], color='lightblue')
plt.plot(range(1,11),result_dict['correct_pairings_basX_mean'], label='qrandK')
#plt.title(f'Correct pairs vs. K - {DATASET_NAME.capitalize()}')
plt.title(f'{DATASET_NAME.capitalize()}')
plt.plot([1,10], [result_dict['correct_pairings_inv_mean']+result_dict['correct_pairings_inv_std'], result_dict['correct_pairings_inv_mean']+result_dict['correct_pairings_inv_std']], color='orange', linestyle = 'dashed')
plt.plot([1,10], [result_dict['correct_pairings_inv_mean']-result_dict['correct_pairings_inv_std'], result_dict['correct_pairings_inv_mean']-result_dict['correct_pairings_inv_std']], color='orange', linestyle = 'dashed')
plt.plot([1,10],[result_dict['correct_pairings_inv_mean'],result_dict['correct_pairings_inv_mean']], label='qinv')
plt.ylim(0.7,1.05)
plt.xlabel('K')
#plt.ylabel('q')
plt.legend()
plt.savefig(os.path.join(PROJ_DIR, 'results', 'plots', f'{DATASET_NAME}_{MODEL_NAME}{GENERATION_MODE}_correct_pairs.png'), dpi=300, bbox_inches='tight')
plt.show()

delta_p = result_dict['correct_pairings_inv_mean']-result_dict['correct_pairings_basX_mean'][0]
print('delta_p', delta_p)

## Plot Kendall's tau

In [ ]:
#plt.figure(figsize=(3,2))
plt.fill_between(range(1,11), result_dict['tau_basX_mean']+result_dict['tau_basX_std'], result_dict['tau_basX_mean']-result_dict['tau_basX_std'], color='lightblue')
plt.plot(range(1,11),result_dict['tau_basX_mean'], label='qrandK')
#plt.title(f'Correct pairs vs. K - {DATASET_NAME.capitalize()}')
plt.title(f'{DATASET_NAME.capitalize()}')
plt.plot([1,10], [result_dict['tau_inv_mean']+result_dict['tau_inv_std'], result_dict['tau_inv_mean']+result_dict['tau_inv_std']], color='orange', linestyle = 'dashed')
plt.plot([1,10], [result_dict['tau_inv_mean']-result_dict['tau_inv_std'], result_dict['tau_inv_mean']-result_dict['tau_inv_std']], color='orange', linestyle = 'dashed')
plt.plot([1,10],[result_dict['tau_inv_mean'],result_dict['tau_inv_mean']], label='qinv')
plt.ylim(0.45,1.05)
plt.xlabel('K')
plt.ylabel('$\\tau$')
plt.legend()
plt.savefig(os.path.join(PROJ_DIR, 'results', 'plots', f'{DATASET_NAME}_{MODEL_NAME}{GENERATION_MODE}_tau.png'), dpi=300, bbox_inches='tight')
plt.show()

delta_tau = result_dict['tau_inv_mean']-result_dict['tau_basX_mean'][0]
print('delta_tau', delta_tau)

## Plot Spearman correlation

In [ ]:
#plt.figure(figsize=(3,2))
plt.fill_between(range(1,11), result_dict['spearman_basX_mean']+result_dict['spearman_basX_std'], result_dict['spearman_basX_mean']-result_dict['spearman_basX_std'], color='lightblue')
plt.plot(range(1,11),result_dict['spearman_basX_mean'], label='qrandK')
#plt.title('Spearman correlation vs. K')
plt.title(f'{DATASET_NAME.capitalize()}')
plt.plot([1,10], [result_dict['spearman_inv_mean']+result_dict['spearman_inv_std'], result_dict['spearman_inv_mean']+result_dict['spearman_inv_std']], color='orange', linestyle = 'dashed')
plt.plot([1,10], [result_dict['spearman_inv_mean']-result_dict['spearman_inv_std'], result_dict['spearman_inv_mean']-result_dict['spearman_inv_std']], color='orange', linestyle = 'dashed')
plt.plot([1,10],[result_dict['spearman_inv_mean'],result_dict['spearman_inv_mean']], label='qinv')
plt.ylim(0.6,1.05)
plt.xlabel('K')
#plt.ylabel('q')
plt.legend()
plt.savefig(os.path.join(PROJ_DIR, 'results', 'plots', f'{DATASET_NAME}_{MODEL_NAME}{GENERATION_MODE}_spearman.png'), dpi=300, bbox_inches='tight')
plt.show()

delta_rho = result_dict['spearman_inv_mean']-result_dict['spearman_basX_mean'][0]
print('delta_rho', delta_rho)

## Plot ability to detect exceptional rankings

In [ ]:
boundaries = [0.5, 1, 1.5, 2, 2.5]#, 3, 3.5]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
for i, ax in enumerate(axes.flat):
    ax.fill_between(range(1,11), result_dict['aucs_basX_mean'][:,i+1]+result_dict['aucs_basX_std'][:,i+1], result_dict['aucs_basX_mean'][:,i+1]-result_dict['aucs_basX_std'][:,i+1], color='lightblue')
    ax.plot(range(1,11),result_dict['aucs_basX_mean'][:,i+1], label='qrandX')
    ax.set_xlabel('K')
    counts_str = '|'.join(map(lambda x:f'{x:.2f}',counts_by_sigma_level[:, i]))
    if i<len(boundaries)-2:
        #ax.set_title(f'[{boundaries[i+1]},{boundaries[i+1+1]}]σ ({counts_str})', fontsize=10)
        ax.set_title(f'[{boundaries[i+1]},{boundaries[i+1+1]}]σ', fontsize=16)
    else:
        #ax.set_title(f'[{boundaries[i+1]},inf)σ ({counts_str})', fontsize=10)
        ax.set_title(f'[{boundaries[i+1]},inf)σ', fontsize=16)
    if i>0:
        #ax.set_yticks([])
        ax.set_yticklabels([])
    ax.set_xticks([2,4,6,8,10])
    ax.plot([1,10], [result_dict['aucs_inv_mean'][i+1]+result_dict['aucs_inv_std'][i+1], result_dict['aucs_inv_mean'][i+1]+result_dict['aucs_inv_std'][i+1]], color='orange', linestyle = 'dashed')
    ax.plot([1,10], [result_dict['aucs_inv_mean'][i+1]-result_dict['aucs_inv_std'][i+1], result_dict['aucs_inv_mean'][i+1]-result_dict['aucs_inv_std'][i+1]], color='orange', linestyle = 'dashed')
    ax.plot([1,10],[result_dict['aucs_inv_mean'][i+1],result_dict['aucs_inv_mean'][i+1]], label='qinv')

    if i==3:
        ax.legend(loc='lower right')
    ax.set_ylim(0.8,1.05)
plt.tight_layout()
#plt.suptitle('AUC vs. K')
plt.suptitle(f'{DATASET_NAME.capitalize()}')
plt.savefig(os.path.join(PROJ_DIR, 'results', 'plots', f'{DATASET_NAME}_{MODEL_NAME}{GENERATION_MODE}_auc_exceptional.png'), dpi=300, bbox_inches='tight')
plt.show()
plt.close(fig)

## Plot Spearman correlation for exceptional rankings

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(16, 3))
for i, ax in enumerate(axes.flat):
    ax.fill_between(range(1,11), result_dict['spearman_exceptional_basX_mean'][:,i+1]+result_dict['spearman_exceptional_basX_std'][:,i+1], result_dict['spearman_exceptional_basX_mean'][:,i+1]-result_dict['spearman_exceptional_basX_std'][:,i+1], color='lightblue')
    ax.plot(range(1,11),result_dict['spearman_exceptional_basX_mean'][:,i+1], label='qrandX')
    ax.set_xlabel('K')
    counts_str = '|'.join(map(lambda x:f'{x:.2f}',counts_by_sigma_level[:, i]))
    if i<len(boundaries)-2:
        ax.set_title(f'[{boundaries[i+1]},{boundaries[i+1+1]}]σ', fontsize=16)
    else:
        ax.set_title(f'[{boundaries[i+1]},inf)σ', fontsize=16)
    if i>0:
        #ax.set_yticks([])
        ax.set_yticklabels([])
    ax.plot([1,10], [result_dict['spearman_exceptional_inv_mean'][i+1]+result_dict['spearman_exceptional_inv_std'][i+1], result_dict['spearman_exceptional_inv_mean'][i+1]+result_dict['spearman_exceptional_inv_std'][i+1]], color='orange', linestyle = 'dashed')
    ax.plot([1,10], [result_dict['spearman_exceptional_inv_mean'][i+1]-result_dict['spearman_exceptional_inv_std'][i+1], result_dict['spearman_exceptional_inv_mean'][i+1]-result_dict['spearman_exceptional_inv_std'][i+1]], color='orange', linestyle = 'dashed')
    ax.plot([1,10],[result_dict['spearman_exceptional_inv_mean'][i+1],result_dict['spearman_exceptional_inv_mean'][i+1]], label='qinv')
    ax.set_ylim(0,1.05)
    ax.set_xticks([2,4,6,8,10])
    if i==3:
        ax.legend(loc='lower right')
plt.suptitle('Spearman correlation vs. K')
plt.suptitle(f'{DATASET_NAME.capitalize()}')
plt.savefig(os.path.join(PROJ_DIR, 'results', 'plots', f'{DATASET_NAME}_{MODEL_NAME}{GENERATION_MODE}_spearman_exceptional.png'), dpi=300, bbox_inches='tight')
plt.show()

## Plot Kendall's tau correlation for exceptional rankings

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(16, 3))
for i, ax in enumerate(axes.flat):
    ax.fill_between(range(1,11), result_dict['tau_exceptional_basX_mean'][:,i+1]+result_dict['tau_exceptional_basX_std'][:,i+1], result_dict['tau_exceptional_basX_mean'][:,i+1]-result_dict['tau_exceptional_basX_std'][:,i+1], color='lightblue')
    ax.plot(range(1,11),result_dict['tau_exceptional_basX_mean'][:,i+1], label='qrandX')
    ax.set_xlabel('K')
    if i == 0:
        ax.set_ylabel('$\\tau$')
    counts_str = '|'.join(map(lambda x:f'{x:.2f}',counts_by_sigma_level[:, i]))
    if i<len(boundaries)-2:
        ax.set_title(f'[{boundaries[i+1]},{boundaries[i+1+1]}]σ', fontsize=16)
    else:
        ax.set_title(f'[{boundaries[i+1]},inf)σ', fontsize=16)
    if i>0:
        #ax.set_yticks([])
        ax.set_yticklabels([])
    ax.plot([1,10], [result_dict['tau_exceptional_inv_mean'][i+1]+result_dict['tau_exceptional_inv_std'][i+1], result_dict['tau_exceptional_inv_mean'][i+1]+result_dict['tau_exceptional_inv_std'][i+1]], color='orange', linestyle = 'dashed')
    ax.plot([1,10], [result_dict['tau_exceptional_inv_mean'][i+1]-result_dict['tau_exceptional_inv_std'][i+1], result_dict['tau_exceptional_inv_mean'][i+1]-result_dict['tau_exceptional_inv_std'][i+1]], color='orange', linestyle = 'dashed')
    ax.plot([1,10],[result_dict['tau_exceptional_inv_mean'][i+1],result_dict['tau_exceptional_inv_mean'][i+1]], label='qinv')
    ax.set_ylim(0,1.05)
    ax.set_xticks([2,4,6,8,10])
    if i==3:
        ax.legend(loc='lower right')
plt.suptitle('Kendall\'s tau correlation vs. K')
plt.suptitle(f'{DATASET_NAME.capitalize()}')
plt.savefig(os.path.join(PROJ_DIR, 'results', 'plots', f'{DATASET_NAME}_{MODEL_NAME}{GENERATION_MODE}_tau_exceptional.png'), dpi=300, bbox_inches='tight')
plt.show()